# Some deeper Data Exploration and Data Selection
This notebook takes a deeper look at the Mercedes-Benz Kaggle data. Competion @: https://www.kaggle.com/c/mercedes-benz-greener-manufacturing 
Use of data subject to conditions mentioned in above link.
After initial Exploration and crude experimenting with models, this notebook looks at the data from a different point of view: The y variable is treated as the average per feature, so the actual time would be y * qty_features. In this notebook, we'll predict this y_actual and then process it to get the y per feature. Further, we'll have a look at the correlation of features, trying to reduce the quantity of features to consider. For this we'll start with a sub-set of the categorical data, and add the out of the 40 lowest correlation factor features the 4 that create the highest rating. remove those 4 out of the pool and repeat until no more improvement can be observed. I'd expect the maximum number to be either around 80 or 120, based on first crude model explorations.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import re
import seaborn as sns
import operator
import string
from itertools import combinations
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.metrics import r2_score
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
#from sklearn.decomposition import PCA
#from sklearn.neural_network import MLPRegressor
#from IPython.display import clear_output


## 1. Load Data Sets

In [2]:
df_train = pd.read_csv('../data/train.csv').drop(['ID'],1).fillna(0)
#df_train = df_train[df_train['y'] < 200] # drop one outlier data (we'll se about this first)
df_test = pd.read_csv('../data/test.csv', index_col = 'ID').fillna(0)

## 2. Construct additional features and change letter categorical to continous

In [3]:
li_columnNames = [e for e in list(df_train.columns.values) if e not in {'y', 'X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8'}]
list_value = list(string.lowercase) + list('a' + x for x in list(string.lowercase)) + ['ba', 'bb', 'bc']
i = 0
value_to_int = {}
for j in list_value:
    value_to_int[j] = i
    i += 1
df_train_eng = df_train.replace(value_to_int).copy()
df_test_eng = df_test.replace(value_to_int).copy()
df_train_eng['qty_options'] = df_train_eng[li_columnNames].sum(axis=1)
df_test_eng['qty_options'] = df_test_eng[li_columnNames].sum(axis=1)
df_train_eng['y_actual'] = df_train_eng['y'] * df_train_eng['qty_options']
df_train_eng[['y', 'qty_options', 'y_actual']].head(5)

,y,qty_options,y_actual
0,130.81,52,6802.12
1,88.53,52,4603.56
2,76.26,73,5566.98
3,80.62,62,4998.44
4,78.02,58,4525.16


## 3. First let's see how linear regression does with the y_actual:

In [4]:
list_index = li_columnNames
list_test = ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8', 'qty_options']
list_test = list_test + list_index
list_train = ['y_actual'] + list_test
df_X = df_train_eng[list_train]
df_Kaggle = df_test_eng[list_test]
df_y = df_X['y_actual']
df_X = df_X.drop('y_actual', axis=1)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.3, random_state=0)

In [6]:
model = LinearRegression(fit_intercept=False, normalize = True)

In [7]:
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=1, normalize=True)

In [8]:
y_pred = pd.Series(model.predict(X_test), index=X_test.index)
#y_Kaggle = pd.Series(linearRegress.predict(df_Kaggle), name='y', index=df_Kaggle.index)
r2_score(y_test, y_pred)

-23096634773826740.0

## How about Ridge CV?

In [9]:
model = RidgeCV()

In [10]:
model.fit(X_train, y_train)
y_pred = pd.Series(model.predict(X_test), index=X_test.index)
print r2_score(y_test, y_pred)
y_Kaggle = pd.Series(model.predict(df_Kaggle), name='y_actual', index=df_Kaggle.index) / df_Kaggle['qty_options']
y_Kaggle.to_csv(path='../data/201706101100.csv', index_label='ID', header = True)

0.79076936979


Ok, let's see that again with a 90-10 data split:

In [11]:
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.1, random_state=0)
model.fit(X_train, y_train)
y_pred = pd.Series(model.predict(X_test), index=X_test.index)
print r2_score(y_test, y_pred)
y_Kaggle = pd.Series(model.predict(df_Kaggle), name='y_actual', index=df_Kaggle.index) / df_Kaggle['qty_options']
#y_Kaggle.to_csv(path='../data/201706101100.csv', index_label='ID', header = True)

0.695278704374


Looks like we're overfitting. Not surprisingwith #festures versus #datasets. So let's start getting creative:
1. let's limit to the categorical data and find the maximum possible solution
2. find the 40 lowest correlated binary features and add the 4-tuplet that creates the maximum increase in r^2. repeat until no more iincrease

In [12]:
list_test = ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8', 'qty_options']
output_test = sum([map(list, combinations(list_test, i)) for i in range(len(list_test) + 1)], [])
best_r2 = -1E+80
best_values = {}
for i in range(1, len(output_test)):
    output_train = ['y_actual'] + output_test[i]
    df_X = df_train_eng[output_train]
    df_y = df_X['y_actual']
    df_X = df_X.drop('y_actual', axis=1)
    X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.1, random_state=0)
    model.fit(X_train, y_train)
    y_pred = pd.Series(model.predict(X_test), index=X_test.index)
    current_r2 = r2_score(y_test, y_pred)
    if current_r2 > best_r2:
        best_r2 = current_r2; best_values['r2'] = current_r2; best_values['features'] = output_test[i]
        print 'New best R^2 score: ', best_r2, ' with features: ', best_values['features']    
print 'Best R^2 score: ', best_r2, ' with features: ', best_values['features']  
    
#    y_Kaggle = pd.Series(model.predict(df_Kaggle), name='y_actual', index=df_Kaggle.index) / df_Kaggle['qty_options']

New best R^2 score:  9.00982716923e-05  with features:  ['X0']
New best R^2 score:  0.0119037568823  with features:  ['X1']
New best R^2 score:  0.0439569050808  with features:  ['X2']
New best R^2 score:  0.0974327145926  with features:  ['X3']
New best R^2 score:  0.520952396186  with features:  ['qty_options']
New best R^2 score:  0.523609475774  with features:  ['X0', 'qty_options']
New best R^2 score:  0.533029212575  with features:  ['X2', 'qty_options']
New best R^2 score:  0.535601219992  with features:  ['X0', 'X2', 'qty_options']
New best R^2 score:  0.541084981835  with features:  ['X2', 'X3', 'qty_options']
New best R^2 score:  0.544095738776  with features:  ['X0', 'X2', 'X3', 'qty_options']
New best R^2 score:  0.544096935081  with features:  ['X0', 'X2', 'X3', 'X8', 'qty_options']
Best R^2 score:  0.544096935081  with features:  ['X0', 'X2', 'X3', 'X8', 'qty_options']


This is the result with only the lower 10. How des this differ when including all binary?

In [13]:
list_index = [e for e in list(df_train.columns.values) if e not in {'y_actual', 'y', 'X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8'}]
list_test = ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8', 'qty_options']
output = sum([map(list, combinations(list_test, i)) for i in range(len(list_test) + 1)], [])
best_r2 = -1E+80
best_values = {}
for i in range(1, len(output)):
    output_test = output[i] + list_index
    output_train = ['y_actual'] + output_test
    df_X = df_train_eng[output_train]
    df_y = df_X['y_actual']
    df_X = df_X.drop('y_actual', axis=1)
    X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.1, random_state=0)
    model = RidgeCV()
    model.fit(X_train, y_train)
    y_pred = pd.Series(model.predict(X_test), index=X_test.index)
    current_r2 = r2_score(y_test, y_pred)
    if current_r2 > best_r2:
        best_r2 = current_r2; best_values['r2'] = current_r2; best_values['features'] = output[i]
        print 'New best R^2 score: ', best_r2, ' with features: ', best_values['features']    
print 'Best R^2 score: ', best_r2, ' with features: ', best_values['features']  
output_test = best_values['features'] + list_index
output_train = ['y_actual'] + output_test
df_X = df_train_eng[output_train]
df_y = df_X['y_actual']
df_X = df_X.drop('y_actual', axis=1)
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.1, random_state=0)
model = RidgeCV()
model.fit(X_train, y_train)
X_Kaggle = df_test_eng[output_test]
y_Kaggle = pd.Series(model.predict(X_Kaggle) / df_test_eng['qty_options'], name='y', index=df_Kaggle.index)
y_Kaggle.to_csv(path='../data/201706111125.csv', index_label='ID', header = True)

New best R^2 score:  0.695373986091  with features:  ['X0']
New best R^2 score:  0.695489035505  with features:  ['X2']
New best R^2 score:  0.695510691756  with features:  ['X5']
New best R^2 score:  0.695529901643  with features:  ['X6']
New best R^2 score:  0.695850758259  with features:  ['qty_options']
New best R^2 score:  0.69588548315  with features:  ['X0', 'qty_options']
New best R^2 score:  0.695984567947  with features:  ['X2', 'qty_options']
New best R^2 score:  0.696018348226  with features:  ['X6', 'qty_options']
New best R^2 score:  0.696041047614  with features:  ['X0', 'X6', 'qty_options']
New best R^2 score:  0.696120158063  with features:  ['X2', 'X5', 'qty_options']
New best R^2 score:  0.696149695324  with features:  ['X2', 'X6', 'qty_options']
New best R^2 score:  0.696155462427  with features:  ['X5', 'X6', 'qty_options']
New best R^2 score:  0.696169534156  with features:  ['X0', 'X2', 'X6', 'qty_options']
New best R^2 score:  0.696192358655  with features:  ['X

Ok, looks like X0, X2, X3 and qty_options are common. Let's take those and add the rest to the "big pool"

## First some data clensing / Identifying data for possible clensing

### Identify duplicate columns

In [5]:
all_data = pd.concat((df_train, df_test))

# remove duplicated columns
c = all_data.columns
print('\n Number of columns before cleaning: %d' % len(c))
li_duplicates = []
for i in range(len(c)-1):
    v = all_data[c[i]].values
    for j in range(i+1,len(c)):
        if np.array_equal(v,all_data[c[j]].values):
            li_duplicates.append(c[j])
            print(' Column %s is identical to %s. Removing %s' % (str(c[i]), str(c[j]), str(c[j])))
li_duplicates = list(set(li_duplicates))
print '\n Number of identified duplicates marked: %s' % len(li_duplicates)


 Number of columns before cleaning: 377
 Column X102 is identical to X214. Removing X214
 Column X102 is identical to X239. Removing X239
 Column X102 is identical to X53. Removing X53
 Column X112 is identical to X199. Removing X199
 Column X113 is identical to X134. Removing X134
 Column X113 is identical to X147. Removing X147
 Column X113 is identical to X222. Removing X222
 Column X113 is identical to X48. Removing X48
 Column X118 is identical to X119. Removing X119
 Column X125 is identical to X227. Removing X227
 Column X134 is identical to X147. Removing X147
 Column X134 is identical to X222. Removing X222
 Column X134 is identical to X48. Removing X48
 Column X138 is identical to X146. Removing X146
 Column X147 is identical to X222. Removing X222
 Column X147 is identical to X48. Removing X48
 Column X152 is identical to X226. Removing X226
 Column X152 is identical to X326. Removing X326
 Column X155 is identical to X360. Removing X360
 Column X17 is identical to X382. Re

### Identify complementary columns

In [6]:
list_index = [e for e in list(df_train.columns.values) if e not in {'y_actual', 'y', 'X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8'}]
all_data = pd.concat((df_train[list_index], df_test[list_index]))
value_compliment = {0: 1, 1: 0}

# remove complimentary columns
c = all_data.columns
print('\n Number of columns before cleaning: %d' % len(c))
li_compliments = []
for i in range(len(c)-1):
    v = all_data[c[i]].replace(value_compliment).values
    for j in range(i+1,len(c)):
        if np.array_equal(v,all_data[c[j]].values):
            li_compliments.append(c[j])
            print(' Column %s is complimentary to %s. Removing %s' % (str(c[i]), str(c[j]), str(c[j])))
li_compliments = list(set(li_compliments))
print '\n Number of identified complements marked: %s' % len(li_compliments)


 Number of columns before cleaning: 368
 Column X128 is complimentary to X130. Removing X130
 Column X156 is complimentary to X157. Removing X157
 Column X204 is complimentary to X205. Removing X205
 Column X232 is complimentary to X263. Removing X263
 Column X263 is complimentary to X279. Removing X279

 Number of identified complements marked: 5


In [7]:
#list_restart = []
#list_restart = ['X46', 'X64', 'X29', 'X28', 'X20', 'X22', 'X256', 'X158', 'X150', 'X118', 'X136', 'X111', 'X115', 'X334', 'X75', 'X264', 'X166', 'X163', 'X148', 'X224', 'X100', 'X103', 'X209', 'X108', 'X314', 'X337', 'X129', 'X313', 'X127', 'X265', 'X261', 'X120']
list_restart = ['X46', 'X355', 'X189', 'X64', 'X29', 'X28', 'X20', 'X22', 'X171', 'X256', 'X158', 'X154', 'X150', 'X118', 'X136', 'X111', 'X116', 'X115', 'X129', 'X75', 'X127', 'X100', 'X163', 'X148', 'X224', 'X166', 'X103', 'X209', 'X283', 'X202', 'X108', 'X201', 'X314', 'X337', 'X334', 'X313', 'X264', 'X265', 'X261', 'X120']
list_test = ['X0', 'X2', 'X3', 'qty_options']
list_exclude = ['y'] + li_compliments + li_duplicates
best_r2 = -1E+80
best_values = {}
best_values['new_features'] = list_restart
best_values['features'] = list_restart
interm_r2 = -1.1E+80
delta = 0.0000000001
while best_r2 - interm_r2 > delta:
#    list_exclude = list_exclude + best_values['new_features']
    list_test = list_test + best_values['new_features']
    best_values['r2'] = best_r2
    list_remainder = [e for e in list(df_train_eng.columns.values) if e not in list_exclude]
    list_corr = list_remainder + ['y_actual']
    corr = df_train_eng[list_remainder].corr().fillna(-2)
    corr['calc'] = corr['y_actual']
    for i in list_test:
        corr['calc'] = corr['calc'] * corr[i]
    li_candidates = [e for e in corr['calc'].sort_values(ascending=True)[0:36 + len(list_test)].index.tolist() \
                     if e not in list(list_test + list_exclude + ['y_actual'])]
    print 'Calculating new correlation matrix'
    output = sum([map(list, combinations(li_candidates, i)) for i in range(1,5)], [])
    for i in range(1, len(output)):
        output_test = output[i] + list_test
#        print output_test
        output_train = ['y_actual'] + output_test
        df_X = df_train_eng[output_train]
        df_y = df_X['y_actual']
        df_X = df_X.drop('y_actual', axis=1)
        X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.1, random_state=0)
        model = RidgeCV()
        model.fit(X_train, y_train)
        y_pred = pd.Series(model.predict(X_test), index=X_test.index)
        current_r2 = r2_score(y_test, y_pred)
        if i % 100 == 0:
            print '.',
        if current_r2 > best_r2:
            interm_r2 = best_r2
            best_r2 = current_r2
            best_values['new_features'] = output[i]
            print '\n New best R^2 score: ', best_r2, ' with features: ', best_values['new_features']
    best_values['features'] = best_values['features'] + best_values['new_features'] 
    print '\n Best R^2 score: ', best_r2, ' with features: ', list(set(best_values['features']))
print '\n Final R^2 score: ', best_r2, ' with features: ', list(set(best_values['features']))
output_test = best_values['features'] + list_index
output_train = ['y_actual'] + output_test
df_X = df_train_eng[output_train]
df_y = df_X['y_actual']
df_X = df_X.drop('y_actual', axis=1)
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.1, random_state=0)
model = RidgeCV()
model.fit(X_train, y_train)
X_Kaggle = df_test_eng[output_test]
y_Kaggle = pd.Series(model.predict(X_Kaggle) / df_test_eng['qty_options'], name='y', index=df_Kaggle.index)
#y_Kaggle.to_csv(path='../data/201706111545.csv', index_label='ID', header = True)

Calculating new correlation matrix

 New best R^2 score:  0.707256900291  with features:  ['X289']

 New best R^2 score:  0.707259141093  with features:  ['X238']

 New best R^2 score:  0.707297756032  with features:  ['X223']

 New best R^2 score:  0.707302702821  with features:  ['X275']

 New best R^2 score:  0.707306653786  with features:  ['X170']

 New best R^2 score:  0.707338775692  with features:  ['X376']

 New best R^2 score:  0.707364631117  with features:  ['X327']
. 
 New best R^2 score:  0.707364631118  with features:  ['X290', 'X327']
. . . . . 
 New best R^2 score:  0.707368486491  with features:  ['X238', 'X327']
. . . . 
 New best R^2 score:  0.707383209674  with features:  ['X223', 'X376']

 New best R^2 score:  0.707407171566  with features:  ['X223', 'X327']
. . 
 New best R^2 score:  0.707423434179  with features:  ['X170', 'X327']
. . . . . . 
 New best R^2 score:  0.707502121665  with features:  ['X376', 'X327']
. . . . . . . . . . . . . . . . . . . 
 New best 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/kaggler/anaconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/home/kaggler/anaconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 313, in wrapped
    return f(*args, **kwargs)
  File "/home/kaggler/anaconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 358, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/home/kaggler/anaconda2/lib/python2.7/inspect.py", line 1048, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/home/kaggler/anaconda2/lib/python2.7/inspect.py", line 1008, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/home/kaggler/anaconda2/lib/python2.7/inspect.py", line 453, in getsourcefile
    if hasattr(getmodule(object, filename), '__loader__'):


IndexError: string index out of range

## to-do's:
1. build model and experiment with deleting compliments and identicalls.
2. build algo to optimize Ridge (one submnission per day)
3. build algo to optimize Elastic Net
4. build algo to optimize random forest
5. build aldo to optimize logistic regression
6. build alog to optimize lasso
7. build alog to optimize mlp
8. build algo to combine (ridge)
9. build algo to combine (lasso)
10. build algo to combine (linear)
11. build algo to combine (random forest)
12. build algo to combine (mlp)
13. look into mxnet

### 1. Model (RidgeCV) to experiment with deleting compliments and identicals

In [ ]:
list_index = 
#list_index = [e for e in list_index if e not in li_compliments]
#list_index = [e for e in list_index if e not in li_duplicates]
list_test = ['X0', 'X2', 'X3', 'qty_options']
list_test = list_test + list_index
list_train = ['y_actual'] + list_test

df_X = df_train_eng[list_train]
df_Kaggle = df_test_eng[list_test]
df_y = df_X['y_actual']
df_X = df_X.drop('y_actual', axis=1)

X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.1, random_state=0)

model = RidgeCV()
model.fit(X_train, y_train)

y_pred = pd.Series(model.predict(X_test), index=X_test.index)
print r2_score(y_test, y_pred)

y_Kaggle = pd.Series(model.predict(X_Kaggle) / df_test_eng['qty_options'], name='y', index=df_Kaggle.index)
#y_Kaggle.to_csv(path='../data/1data.201706112000.csv', index_label='ID', header = True)


### 2. Optimize algo RidgeCV

In [ ]:
list_index = 
#list_index = [e for e in list_index if e not in li_compliments]
#list_index = [e for e in list_index if e not in li_duplicates]
list_test = ['X0', 'X2', 'X3', 'qty_options']
list_test = list_test + list_index
list_train = ['y_actual'] + list_test

best_r2 = -1E+80
best_values = {}

df_X = df_train_eng[list_train]
df_Kaggle = df_test_eng[list_test]
df_y = df_X['y_actual']
df_X = df_X.drop('y_actual', axis=1)

X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=0)

for intercept in (True, False):
    model = RidgeCV(alphas=(0.1, 0.3, 0.7, 1.0, 3.0, 7.0, 10.0), fit_intercept = intercept, cv = 5)
    model.fit(X_train, y_train)

    y_pred = pd.Series(model.predict(X_test), index=X_test.index)
    current_r2 =  r2_score(y_test, y_pred)
    if current_r2 > best_r2:
        best_r2 = current_r2; best_values['r2'] = current_r2; best_values['intercept'] = intercept
print 'Best R^2 score: ', best_r2, ' with values: intercept: ', best_values['intercept']

model = RidgeCV(alphas=(0.1, 0.3, 0.7, 1.0, 3.0, 7.0, 10.0), fit_intercept = intercept, cv = 5)
model.fit(X_train, y_train)

y_Kaggle = pd.Series(model.predict(df_Kaggle) / df_test_eng['qty_options'], name='y', index=df_Kaggle.index)
#y_Kaggle.to_csv(path='../data/1data.201706112000.csv', index_label='ID', header = True)


### 3. Optimize algo ElasticNetCV

### 4. Optimize algo Random Forest

In [ ]:
'X46', 'X61', 'X64', 'X66', 'X28', 'X27', 'X20', 'X22', 'X171', 'X81', 'X256', 'X156', 'X150', 'X119', 'X294', 'X276', 'X130', 'X115', 'X129', 'X52', 'X76', 'X75', 'X283', 'X166', 'X163', 'X148', 'X374', 'X100', 'X103', 'X209', 'X321', 'X142', 'X147', 'X201', 'X316', 'X336', 'X337', 'X334', 'X313', 'X265', 'X261', 'X263'
'X54', 'X376', 'X154', 'X108', 'X328', 'X111'
'X64', 'X232', 'X256', 'X150', 'X118', 'X276', 'X136', 'X111', 'X129', 'X75', 'X127', 'X100', 'X224', 'X166', 'X209', 'X108', 'X314', 'X316', 'X337', 'X334', 'X264', 'X265', 'X261', 'X120'
'X46', 'X29', 'X28', 'X22', 'X256', 'X158', 'X150', 'X118', 'X136', 'X111', 'X115', 'X129', 'X75', 'X127', 'X100', 'X163', 'X166', 'X103', 'X209', 'X108', 'X314', 'X337', 'X334', 'X313', 'X264', 'X265', 'X261', 'X120'
'X46', 'X64', 'X29', 'X28', 'X20', 'X22', 'X256', 'X158', 'X150', 'X118', 'X136', 'X111', 'X115', 'X334', 'X75', 'X264', 'X166', 'X163', 'X148', 'X224', 'X100', 'X103', 'X209', 'X108', 'X314', 'X337', 'X129', 'X313', 'X127', 'X265', 'X261', 'X120'
'X46', 'X189', 'X64', 'X29', 'X28', 'X20', 'X22', 'X171', 'X256', 'X158', 'X154', 'X150', 'X118', 'X337', 'X136', 'X111', 'X116', 'X115', 'X376', 'X334', 'X75', 'X264', 'X327', 'X166', 'X163', 'X148', 'X223', 'X224', 'X100', 'X103', 'X209', 'X202', 'X108', 'X201', 'X314', 'X285', 'X283', 'X129', 'X313', 'X127', 'X265', 'X120', 'X355', 'X261'
'X46', 'X189', 'X64', 'X29', 'X28', 'X20', 'X22', 'X171', 'X256', 'X158', 'X154', 'X150', 'X118', 'X337', 'X136', 'X111', 'X116', 'X115', 'X376', 'X50', 'X334', 'X79', 'X75', 'X264', 'X327', 'X166', 'X163', 'X148', 'X223', 'X224', 'X100', 'X103', 'X209', 'X202', 'X108', 'X378', 'X201', 'X314', 'X285', 'X336', 'X283', 'X129', 'X313', 'X127', 'X265', 'X120', 'X355', 'X261'
'X46', 'X189', 'X64', 'X29', 'X28', 'X20', 'X22', 'X255', 'X171', 'X256', 'X158', 'X178', 'X154', 'X150', 'X118', 'X337', 'X136', 'X111', 'X116', 'X115', 'X376', 'X50', 'X334', 'X79', 'X75', 'X264', 'X327', 'X166', 'X163', 'X148', 'X374', 'X223', 'X224', 'X100', 'X103', 'X209', 'X168', 'X202', 'X108', 'X378', 'X201', 'X314', 'X285', 'X336', 'X283', 'X129', 'X313', 'X127', 'X265', 'X120', 'X355', 'X261'
'X46', 'X189', 'X64', 'X29', 'X28', 'X20', 'X22', 'X255', 'X171', 'X256', 'X158', 'X178', 'X154', 'X150', 'X118', 'X337', 'X136', 'X111', 'X116', 'X115', 'X376', 'X50', 'X334', 'X79', 'X75', 'X264', 'X327', 'X166', 'X163', 'X148', 'X374', 'X223', 'X224', 'X100', 'X103', 'X209', 'X168', 'X202', 'X108', 'X378', 'X201', 'X314', 'X285', 'X336', 'X283', 'X129', 'X313', 'X127', 'X265', 'X120', 'X355', 'X261', 'X275', 'X331', 'X352', 'X294'
